In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tdastro.opsim.ztf_opsim import ZTFOpsim, _ztfcam_ccd_gain
from tdastro.astro_utils.mag_flux import mag2flux

In [2]:
opsim_db = ZTFOpsim.from_db("data/ztf_metadata_latest.db")
obs_log = pd.read_parquet('ztfsniadr2/tables/observing_logs.parquet')
obs_log = obs_log.drop_duplicates("expid")
obs_log["filter"] = obs_log.apply(lambda row: row["band"][-1],axis=1)
obs_log = pd.merge(obs_log, opsim_db.table[["expid","filter","exptime","fwhm","obsmjd","obsdate","scibckgnd","ra","dec"]],on=["filter","expid"])
gain = _ztfcam_ccd_gain
obs_log["zp_nJy"] = mag2flux(obs_log["zp"].values + 2.5*np.log10(gain))

/Users/mi/Work/tdastro/tdastro/src/tdastro/opsim/ztf_opsim.py:170: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.table = self.table.replace("", np.nan)


In [3]:
obs_log.to_parquet("data/ztf_observing_log_combined_w_metadata.parquet")